## Monterey Tides Simulations
8760 rows per simulation<br>

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder.

3. Convert from E+ 8.1 to 9.5 using the .lst format.

4. Use E+ template file to change the outputs. See Word doc that describes the process. Process all IDFs.

5. Rerun batch sims in EnergyPlus v9.5.

6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files manually before importing into this notebook.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/CommercialBuildings

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
# State Average
# comgas_cost = 0.95
# comelec_cost = 0.1743

# Provided by client
comgas_cost = 0.62915
comelec_cost = 0.127249

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.07
elec_source = 1.88

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.14311

In [5]:
# Zone size for EUI and cost/sf

sf = 180

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory 

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/monterey/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/monterey/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 105121 rows, 24 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,e-clear,e,clear,01/01 01:00:00,5.875,0.975,33696.00714,0.0,0.0,0.0,...,14.662537,1.00,1.0,10.5,21.055,0,321342.89260,0,322532.0731,0.0
1,e-clear,e,clear,01/01 02:00:00,7.000,0.975,33696.00714,0.0,0.0,0.0,...,14.650341,1.00,1.0,10.5,21.055,0,278331.91470,0,322532.0731,0.0
2,e-clear,e,clear,01/01 03:00:00,7.000,0.975,14256.00302,0.0,0.0,0.0,...,14.755968,1.00,1.0,10.5,21.055,0,300730.43040,0,322532.0731,0.0
3,e-clear,e,clear,01/01 04:00:00,7.000,0.975,14256.00302,0.0,0.0,0.0,...,14.983982,1.00,1.0,10.5,21.055,0,318356.32170,0,322532.0731,0.0
4,e-clear,e,clear,01/01 05:00:00,6.375,0.975,14256.00302,0.0,0.0,0.0,...,15.327302,1.00,1.0,10.5,21.055,0,382539.20650,0,322532.0731,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,w-lowe,w,lowe,12/31 20:00:00,7.375,0.975,129600.02750,0.0,0.0,0.0,...,5.008956,0.25,1.0,10.8,21.148,0,0.00000,0,424386.7498,0.0
8756,w-lowe,w,lowe,12/31 21:00:00,7.000,0.975,129600.02750,0.0,0.0,0.0,...,5.004455,1.00,1.0,10.8,21.148,0,0.00000,0,424386.7498,0.0
8757,w-lowe,w,lowe,12/31 22:00:00,7.625,0.750,129600.02750,0.0,0.0,0.0,...,5.024198,1.00,1.0,10.8,21.148,0,0.00000,0,424386.7498,0.0
8758,w-lowe,w,lowe,12/31 23:00:00,6.125,0.750,110160.02330,0.0,0.0,0.0,...,5.259621,1.00,1.0,10.8,21.148,0,48283.45347,0,424386.7498,0.0


In [10]:
list(all_ComElec.columns.values)

['Scenario',
 'Elevation',
 'GlazingSystem',
 'Date/Time',
 'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
 'COMFENZONE PEOPLE 1:People Occupant Count [](Hourly)',
 'COMFENZONE:Zone Lights Electricity Energy [J](Hourly)',
 'COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly)',
 'COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly)',
 'EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly)',
 'EXTWALL:Surface Outside Face Incident Solar Radiation Rate per Area [W/m2](Hourly)',
 'EXTWALL:Surface Outside Face Incident Beam Solar Radiation Rate per Area [W/m2](Hourly)',
 'EXTWALL:Surface Outside Face Incident Sky Diffuse Solar Radiation Rate per Area [W/m2](Hourly)',
 'WIN_1_1:Surface Outside Face Sunlit Fraction [](Hourly)',
 'WIN_1_1:Surface Outside Face Incident Solar Radiation Rate per Area [W/m2](Hourly)',
 'WIN_1_1:Surface Outside Face Incident Beam Solar Radiation Rate per Area [W/m2](Hourly)',
 'WIN_1_1:Surface Outside Face Incident Sky Diffuse Solar Ra

In [11]:
# Get wanted columns
all_energy = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
             "SYS-1:Air System Fan Electricity Energy [J](Hourly)"]]

# all_energy

In [12]:
all_energy = all_energy.dropna().reset_index(drop=True)

all_energy

,Scenario,Date/Time,Elevation,GlazingSystem,COMFENZONE:Zone Lights Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly)
0,e-clear,01/01 01:00:00,e,clear,33696.00714,0.0,321342.89260,322532.0731
1,e-clear,01/01 02:00:00,e,clear,33696.00714,0.0,278331.91470,322532.0731
2,e-clear,01/01 03:00:00,e,clear,14256.00302,0.0,300730.43040,322532.0731
3,e-clear,01/01 04:00:00,e,clear,14256.00302,0.0,318356.32170,322532.0731
4,e-clear,01/01 05:00:00,e,clear,14256.00302,0.0,382539.20650,322532.0731
...,...,...,...,...,...,...,...,...
105115,w-lowe,12/31 20:00:00,w,lowe,129600.02750,0.0,0.00000,424386.7498
105116,w-lowe,12/31 21:00:00,w,lowe,129600.02750,0.0,0.00000,424386.7498
105117,w-lowe,12/31 22:00:00,w,lowe,129600.02750,0.0,0.00000,424386.7498
105118,w-lowe,12/31 23:00:00,w,lowe,110160.02330,0.0,48283.45347,424386.7498


In [13]:
all_energy_group = all_energy.groupby("Scenario")

# all_energy_group.sum()

In [14]:
# Export grouped file to csv
all_energy.to_csv("Scraper_Output/monterey_all.csv", header=True, index=True)

In [15]:
# # Export grouped file to csv
# all_energy_group.to_csv("Scraper_Output/msp2_group.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [16]:
# Rename Columns
all_energy2 = all_energy.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

# all_energy2

In [17]:
# list(all_energy2.columns.values)

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [18]:
# Copy dataframe to modify and leave original df intact
get_annual = all_energy2.copy(deep=True)

# get_annual

In [19]:
# list(get_annual.columns.values)

In [20]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual["LightingEnergy(kBtu)"] = (get_annual["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

# get_annual

In [21]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)",  "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

# get_annual

In [22]:
# Add energy totals
totalEnergy = get_annual["FanEnergy(kBtu)"] + get_annual["HeatingEnergy(kBtu)"] + get_annual["LightingEnergy(kBtu)"] + get_annual["CoolingEnergy(kBtu)"]
        
get_annual["AnnualEnergy(kBtu)"] = totalEnergy

# get_annual

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [23]:
# Get data per sf (energy/sf)
get_annual["FanEnergy(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"]) / sf
get_annual["HeatingEnergy(kBtu/sf)"] = (get_annual["HeatingEnergy(kBtu)"]) / sf
get_annual["CoolingEnergy(kBtu/sf)"] = (get_annual["CoolingEnergy(kBtu)"]) / sf
get_annual["LightingEnergy(kBtu/sf)"] = (get_annual["LightingEnergy(kBtu)"]) / sf

In [24]:
# Establish EUI (energy/sf)
get_annual["EUI(kBtu/sf)"] = (get_annual["FanEnergy(kBtu)"] +
                                    get_annual["HeatingEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +  
                                    get_annual["CoolingEnergy(kBtu)"]) / sf
                                                       
# get_annual

-----

## Get Carbon per sf 
Add carbon/sf column

In [25]:
# Do carbon calculation using variable (lb/sf)
get_annual["CO2_Electric(lb/sf)"] = ((get_annual["FanEnergy(kBtu)"] +
                                    get_annual["LightingEnergy(kBtu)"] +
                                    get_annual["HeatingEnergy(kBtu)"]+
                                    get_annual["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
get_annual

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf)
0,e-clear,01/01 01:00:00,e,clear,0.305701,0.304574,0.0,0.031938,0.642213,0.001698,0.001692,0.0,0.000177,0.003568,0.000511
1,e-clear,01/01 02:00:00,e,clear,0.305701,0.263808,0.0,0.031938,0.601447,0.001698,0.001466,0.0,0.000177,0.003341,0.000478
2,e-clear,01/01 03:00:00,e,clear,0.305701,0.285037,0.0,0.013512,0.604251,0.001698,0.001584,0.0,0.000075,0.003357,0.000480
3,e-clear,01/01 04:00:00,e,clear,0.305701,0.301744,0.0,0.013512,0.620957,0.001698,0.001676,0.0,0.000075,0.003450,0.000494
4,e-clear,01/01 05:00:00,e,clear,0.305701,0.362577,0.0,0.013512,0.681791,0.001698,0.002014,0.0,0.000075,0.003788,0.000542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,w-lowe,12/31 20:00:00,w,lowe,0.402241,0.000000,0.0,0.122837,0.525078,0.002235,0.000000,0.0,0.000682,0.002917,0.000417
105116,w-lowe,12/31 21:00:00,w,lowe,0.402241,0.000000,0.0,0.122837,0.525078,0.002235,0.000000,0.0,0.000682,0.002917,0.000417
105117,w-lowe,12/31 22:00:00,w,lowe,0.402241,0.000000,0.0,0.122837,0.525078,0.002235,0.000000,0.0,0.000682,0.002917,0.000417
105118,w-lowe,12/31 23:00:00,w,lowe,0.402241,0.045764,0.0,0.104412,0.552416,0.002235,0.000254,0.0,0.000580,0.003069,0.000439


In [26]:
# get_annual["CO2_Gas(lb/sf)"] = ((get_annual["HeatingEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# # get_annual

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [27]:
# Get annual costs
get_annual_costs = all_energy2.copy(deep=True)

In [28]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs["LightingCost($)"] = (get_annual_costs["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux * comelec_cost

#get_annual_costs

In [29]:
# Drop columns
get_annual_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs

In [30]:
# Add cost totals
totalCost = get_annual_costs["FanCost($)"] +  get_annual_costs["HeatingCost($)"]  + get_annual_costs["CoolingCost($)"] + get_annual_costs["LightingCost($)"]
        
get_annual_costs["AnnualCost($)"] = totalCost

# get_annual_costs

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [31]:
# Get data per sf ($/sf)
get_annual_costs["FanCost($/sf)"] = (get_annual_costs["FanCost($)"]) / sf
get_annual_costs["HeatingCost($/sf)"] = (get_annual_costs["HeatingCost($)"]) / sf
get_annual_costs["CoolingCost($/sf)"] = (get_annual_costs["CoolingCost($)"]) / sf
get_annual_costs["LightingCost($/sf)"] = (get_annual_costs["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [32]:
# Establish costs ($/sf)
get_annual_costs["AnnualCost($/sf)"] = (get_annual_costs["FanCost($)"] +
                                    get_annual_costs["HeatingCost($)"] +  
                                    get_annual_costs["CoolingCost($)"] +  
                                    get_annual_costs["LightingCost($)"]) / sf
                                                       
# get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [33]:
# # Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

# allAnnualData

In [34]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [35]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualData.sum()

In [36]:
annualDataFinal = annualData.sum()

# annualDataFinal

In [37]:
# list(annualDataFinal.columns.values)

In [38]:
# Export grouped file to csv
annualDataFinal.to_csv("Scraper_Output/monterey_annual_com.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [39]:
# Define the scenario for the existing condition.
baselineAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "exist"]

# baselineAnnualData

In [40]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualData = baselineAnnualData.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'B_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'B_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingCost($)':'B_HeatingCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingCost($/sf)':'B_HeatingCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

In [41]:
# Reset index
allBaselineAnnualData.reset_index(inplace=True, drop=True)

# allBaselineAnnualData

In [42]:
# # Define the scenario for the baselines.

clearAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "clear"]

# clearAnnualData

In [43]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualData = clearAnnualData.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'C_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'C_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingCost($)':'C_HeatingCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingCost($/sf)':'C_HeatingCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualData

In [44]:
# Reset index
allClearAnnualData.reset_index(inplace=True, drop=True)

# allClearAnnualData

In [45]:
# Define the scenario for the baselines.

loweAnnualData = allAnnualData[allAnnualData["GlazingSystem"] == "lowe"]

# loweAnnualData

In [46]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualData = loweAnnualData.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingEnergy(kBtu)':'L_HeatingEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingEnergy(kBtu/sf)':'L_HeatingEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingCost($)':'L_HeatingCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingCost($/sf)':'L_HeatingCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [47]:
#Reset index
allLoweAnnualData.reset_index(inplace=True, drop=True)

# allLoweAnnualData

---
## Merge the 3 dataframes to perform delta from existing calculation


In [48]:
# allBaselineAnnualData
# allClearAnnualData

In [49]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualData, allClearAnnualData, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualData, left_index=True, right_index=True)

# AllMergedAnnualData

In [50]:
# list(AllMergedAnnualData.columns.values)

In [51]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
# AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["C_HeatingCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["C_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [52]:
# AllMergedAnnualData

In [53]:
# list(AllMergedAnnualData.columns.values)

In [54]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
# AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($)"] = AllMergedAnnualData["B_HeatingCost($)"] - AllMergedAnnualData["L_HeatingCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingCost($/sf)"] = AllMergedAnnualData["B_HeatingCost($/sf)"] - AllMergedAnnualData["L_HeatingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [55]:
# list(AllMergedAnnualData.columns.values)

In [56]:
# AllMergedAnnualData

---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [57]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=False)

# AnnualDataDelta.sum()

In [58]:
# list(AnnualDataDelta.sum().columns.values)

In [59]:
AnnualDataDelta = AnnualDataDelta.sum()

In [60]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/monterey_deltas_annual_com.csv", header=True, index=False)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [61]:
# Copy dataframe to modify and leave original df intact
get_peak = all_energy2.copy(deep=True)

# get_peak

In [62]:
# list(get_peak.columns.values)

In [63]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalPeakElec = (get_peak["FanEnergy[J](Hourly)"]  + get_peak["CoolingEnergy[J](Hourly)"] + get_peak["LightingEnergy[J](Hourly)"])
totalPeakGas =  get_peak["HeatingEnergy[J](Hourly)"]
        
get_peak["PeakElecEnergy[J](Hourly)"] = totalPeakElec
get_peak["PeakGasEnergy[J](Hourly)"] = totalPeakGas


get_peak["PeakElecEnergy(W)"] = (get_peak["PeakElecEnergy[J](Hourly)"] / 3600)
get_peak["PeakElecEnergy(W/sf)"] = (get_peak["PeakElecEnergy(W)"] / sf)


get_peak["PeakGasEnergy(W)"] = (get_peak["PeakGasEnergy[J](Hourly)"] / 3600)
get_peak["PeakGasEnergy(W/sf)"] = (get_peak["PeakGasEnergy(W)"] / sf)

get_peak

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,e-clear,01/01 01:00:00,e,clear,33696.00714,0.0,321342.89260,322532.0731,356228.08024,321342.89260,98.952245,0.549735,89.261915,0.495900
1,e-clear,01/01 02:00:00,e,clear,33696.00714,0.0,278331.91470,322532.0731,356228.08024,278331.91470,98.952245,0.549735,77.314421,0.429525
2,e-clear,01/01 03:00:00,e,clear,14256.00302,0.0,300730.43040,322532.0731,336788.07612,300730.43040,93.552243,0.519735,83.536231,0.464090
3,e-clear,01/01 04:00:00,e,clear,14256.00302,0.0,318356.32170,322532.0731,336788.07612,318356.32170,93.552243,0.519735,88.432312,0.491291
4,e-clear,01/01 05:00:00,e,clear,14256.00302,0.0,382539.20650,322532.0731,336788.07612,382539.20650,93.552243,0.519735,106.260891,0.590338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,w-lowe,12/31 20:00:00,w,lowe,129600.02750,0.0,0.00000,424386.7498,553986.77730,0.00000,153.885216,0.854918,0.000000,0.000000
105116,w-lowe,12/31 21:00:00,w,lowe,129600.02750,0.0,0.00000,424386.7498,553986.77730,0.00000,153.885216,0.854918,0.000000,0.000000
105117,w-lowe,12/31 22:00:00,w,lowe,129600.02750,0.0,0.00000,424386.7498,553986.77730,0.00000,153.885216,0.854918,0.000000,0.000000
105118,w-lowe,12/31 23:00:00,w,lowe,110160.02330,0.0,48283.45347,424386.7498,534546.77310,48283.45347,148.485215,0.824918,13.412070,0.074512


In [64]:
# get_peak["PeakGasEnergy[J](Hourly)"].idxmax()

In [65]:
# get_peak["PeakElecEnergy[J](Hourly)"].idxmax()

In [66]:
# Get scenario
Eexist = get_peak[get_peak["Scenario"] == "e-exist"]
Eclear = get_peak[get_peak["Scenario"] == "e-clear"]
Elowe = get_peak[get_peak["Scenario"] == "e-lowe"]
Nexist = get_peak[get_peak["Scenario"] == "n-exist"]
Nclear = get_peak[get_peak["Scenario"] == "n-clear"]
Nlowe = get_peak[get_peak["Scenario"] == "n-lowe"]
Sexist = get_peak[get_peak["Scenario"] == "s-exist"]
Sclear = get_peak[get_peak["Scenario"] == "s-clear"]
Slowe = get_peak[get_peak["Scenario"] == "s-lowe"]
Wexist = get_peak[get_peak["Scenario"] == "w-exist"]
Wclear = get_peak[get_peak["Scenario"] == "w-clear"]
Wlowe = get_peak[get_peak["Scenario"] == "w-lowe"]

Eclear

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,e-clear,01/01 01:00:00,e,clear,33696.00714,0.0,321342.89260,322532.0731,356228.08024,321342.89260,98.952245,0.549735,89.261915,0.495900
1,e-clear,01/01 02:00:00,e,clear,33696.00714,0.0,278331.91470,322532.0731,356228.08024,278331.91470,98.952245,0.549735,77.314421,0.429525
2,e-clear,01/01 03:00:00,e,clear,14256.00302,0.0,300730.43040,322532.0731,336788.07612,300730.43040,93.552243,0.519735,83.536231,0.464090
3,e-clear,01/01 04:00:00,e,clear,14256.00302,0.0,318356.32170,322532.0731,336788.07612,318356.32170,93.552243,0.519735,88.432312,0.491291
4,e-clear,01/01 05:00:00,e,clear,14256.00302,0.0,382539.20650,322532.0731,336788.07612,382539.20650,93.552243,0.519735,106.260891,0.590338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,e-clear,12/31 20:00:00,e,clear,129600.02750,0.0,53729.31866,322532.0731,452132.10060,53729.31866,125.592250,0.697735,14.924811,0.082916
8756,e-clear,12/31 21:00:00,e,clear,129600.02750,0.0,24431.28210,322532.0731,452132.10060,24431.28210,125.592250,0.697735,6.786467,0.037703
8757,e-clear,12/31 22:00:00,e,clear,129600.02750,0.0,35882.43103,322532.0731,452132.10060,35882.43103,125.592250,0.697735,9.967342,0.055374
8758,e-clear,12/31 23:00:00,e,clear,110160.02330,0.0,302690.07140,322532.0731,432692.09640,302690.07140,120.192249,0.667735,84.080575,0.467114


In [67]:
EexistPeakGas = pd.DataFrame(Eexist.loc[Eexist['HeatingEnergy[J](Hourly)'].idxmax()])
EexistPeakGas = EexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKGas = pd.DataFrame(Eclear.loc[Eclear['HeatingEnergy[J](Hourly)'].idxmax()])
EclearPeaKGas = EclearPeaKGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakGas = pd.DataFrame(Elowe.loc[Elowe['HeatingEnergy[J](Hourly)'].idxmax()])
ElowePeakGas = ElowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NexistPeakGas = pd.DataFrame(Nexist.loc[Nexist['HeatingEnergy[J](Hourly)'].idxmax()])
NexistPeakGas = NexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NclearPeakGas = pd.DataFrame(Nclear.loc[Nclear['HeatingEnergy[J](Hourly)'].idxmax()])
NclearPeakGas = NclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NlowePeakGas = pd.DataFrame(Nlowe.loc[Nlowe['HeatingEnergy[J](Hourly)'].idxmax()])
NlowePeakGas = NlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SexistPeakGas = pd.DataFrame(Sexist.loc[Sexist['HeatingEnergy[J](Hourly)'].idxmax()])
SexistPeakGas = SexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SclearPeakGas = pd.DataFrame(Sclear.loc[Sclear['HeatingEnergy[J](Hourly)'].idxmax()])
SclearPeakGas =  SclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SlowePeakGas = pd.DataFrame(Slowe.loc[Slowe['HeatingEnergy[J](Hourly)'].idxmax()])
SlowePeakGas = SlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WexistPeakGas = pd.DataFrame(Wexist.loc[Wexist['HeatingEnergy[J](Hourly)'].idxmax()])
WexistPeakGas = WexistPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WclearPeakGas = pd.DataFrame(Wclear.loc[Wclear['HeatingEnergy[J](Hourly)'].idxmax()])
WclearPeakGas =  WclearPeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WlowePeakGas = pd.DataFrame(Wlowe.loc[Wlowe['HeatingEnergy[J](Hourly)'].idxmax()])
WlowePeakGas = WlowePeakGas.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)


In [68]:
EexistPeakElec = pd.DataFrame(Eexist.loc[Eexist['CoolingEnergy[J](Hourly)'].idxmax()])
EexistPeakElec = EexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
EclearPeaKElec = pd.DataFrame(Eclear.loc[Eclear['CoolingEnergy[J](Hourly)'].idxmax()])
EclearPeaKElec = EclearPeaKElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
ElowePeakElec = pd.DataFrame(Elowe.loc[Elowe['CoolingEnergy[J](Hourly)'].idxmax()])
ElowePeakElec = ElowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NexistPeakElec = pd.DataFrame(Nexist.loc[Nexist['CoolingEnergy[J](Hourly)'].idxmax()])
NexistPeakElec = NexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NclearPeakElec = pd.DataFrame(Nclear.loc[Nclear['CoolingEnergy[J](Hourly)'].idxmax()])
NclearPeakElec = NclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
NlowePeakElec = pd.DataFrame(Nlowe.loc[Nlowe['CoolingEnergy[J](Hourly)'].idxmax()])
NlowePeakElec = NlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SexistPeakElec = pd.DataFrame(Sexist.loc[Sexist['CoolingEnergy[J](Hourly)'].idxmax()])
SexistPeakElec = SexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SclearPeakElec = pd.DataFrame(Sclear.loc[Sclear['CoolingEnergy[J](Hourly)'].idxmax()])
SclearPeakElec =  SclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
SlowePeakElec = pd.DataFrame(Slowe.loc[Slowe['CoolingEnergy[J](Hourly)'].idxmax()])
SlowePeakElec = SlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WexistPeakElec = pd.DataFrame(Wexist.loc[Wexist['CoolingEnergy[J](Hourly)'].idxmax()])
WexistPeakElec = WexistPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WclearPeakElec = pd.DataFrame(Wclear.loc[Wclear['CoolingEnergy[J](Hourly)'].idxmax()])
WclearPeakElec =  WclearPeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)
WlowePeakElec = pd.DataFrame(Wlowe.loc[Wlowe['CoolingEnergy[J](Hourly)'].idxmax()])
WlowePeakElec = WlowePeakElec.swapaxes("index", "columns").reset_index().drop(['index'], axis=1)


In [69]:
PeakDemandElec = EexistPeakElec.append([EclearPeaKElec,
ElowePeakElec,
NexistPeakElec,
NclearPeakElec,
NlowePeakElec,
SexistPeakElec,
SclearPeakElec,
SlowePeakElec,
WexistPeakElec,
WclearPeakElec,
WlowePeakElec
])

# PeakDemandElec

In [70]:
PeakDemandGas = EexistPeakGas.append([EclearPeaKGas,
ElowePeakGas,
NexistPeakGas,
NclearPeakGas,
NlowePeakGas,
SexistPeakGas,
SclearPeakGas,
SlowePeakGas,
WexistPeakGas,
WclearPeakGas,
WlowePeakGas
])

# PeakDemandGas

In [71]:
# # Export dataframe  to csv
# PeakDemandElec.to_csv("Scraper_Output/msp2_peakelec_com.csv", header=True, index=False)
# PeakDemandGas.to_csv("Scraper_Output/msp2_peakgas_com.csv", header=True, index=False)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files


In [72]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "exist"]

# baselinePeakDemandElec

In [73]:
# list(baselinePeakDemandElec.columns.values)

In [74]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'B_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'B_PeakElecEnergy[J](Hourly)',
 'PeakGasEnergy[J](Hourly)':'B_PeakGasEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'B_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'B_PeakElecEnergy(W/sf)',
'PeakGasEnergy(W)':'B_PeakGasEnergy(W)',
 'PeakGasEnergy(W/sf)':'B_PeakGasEnergy(W/sf)'

                            })

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf),B_PeakGasEnergy(W),B_PeakGasEnergy(W/sf)
0,e-exist,10/08 13:00:00,e,exist,42768,1.33422e+06,0,346480,1.72347e+06,0,478.74,2.65967,0,0
0,n-exist,10/08 13:00:00,n,exist,42768,1.11992e+06,0,262874,1.42556e+06,0,395.989,2.19994,0,0
0,s-exist,10/08 11:00:00,s,exist,53136,2.23492e+06,0,406149,2.69421e+06,0,748.392,4.15773,0,0
0,w-exist,10/08 17:00:00,w,exist,42768,2.68781e+06,0,582807,3.31338e+06,0,920.384,5.11324,0,0


In [75]:
#Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

allbaselinePeakDemandElec

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_LightingEnergy[J](Hourly),B_CoolingEnergy[J](Hourly),B_HeatingEnergy[J](Hourly),B_FanEnergy[J](Hourly),B_PeakElecEnergy[J](Hourly),B_PeakGasEnergy[J](Hourly),B_PeakElecEnergy(W),B_PeakElecEnergy(W/sf),B_PeakGasEnergy(W),B_PeakGasEnergy(W/sf)
0,e-exist,10/08 13:00:00,e,exist,42768,1.33422e+06,0,346480,1.72347e+06,0,478.74,2.65967,0,0
1,n-exist,10/08 13:00:00,n,exist,42768,1.11992e+06,0,262874,1.42556e+06,0,395.989,2.19994,0,0
2,s-exist,10/08 11:00:00,s,exist,53136,2.23492e+06,0,406149,2.69421e+06,0,748.392,4.15773,0,0
3,w-exist,10/08 17:00:00,w,exist,42768,2.68781e+06,0,582807,3.31338e+06,0,920.384,5.11324,0,0


In [76]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "clear"]

clearPeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,e-clear,10/08 13:00:00,e,clear,42768,1.13615e+06,0,322532,1.50145e+06,0,417.07,2.31706,0,0
0,n-clear,10/08 13:00:00,n,clear,42768,950548,0,240281,1.2336e+06,0,342.666,1.9037,0,0
0,s-clear,10/08 11:00:00,s,clear,53136,1.93281e+06,0,374662,2.36061e+06,0,655.726,3.64292,0,0
0,w-clear,10/08 17:00:00,w,clear,42768,2.31854e+06,0,524928,2.88624e+06,0,801.732,4.45407,0,0


In [77]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'C_HeatingEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'C_PeakElecEnergy[J](Hourly)',
  'PeakGasEnergy[J](Hourly)':'C_PeakGasEnergy[J](Hourly)',
'PeakElecEnergy(W)':'C_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'C_PeakElecEnergy(W/sf)',
'PeakGasEnergy(W)':'C_PeakGasEnergy(W)',
 'PeakGasEnergy(W/sf)':'C_PeakGasEnergy(W/sf)'
})

# allClearPeakDemandElec

In [78]:
#Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [79]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "lowe"]

lowePeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,LightingEnergy[J](Hourly),CoolingEnergy[J](Hourly),HeatingEnergy[J](Hourly),FanEnergy[J](Hourly),PeakElecEnergy[J](Hourly),PeakGasEnergy[J](Hourly),PeakElecEnergy(W),PeakElecEnergy(W/sf),PeakGasEnergy(W),PeakGasEnergy(W/sf)
0,e-lowe,10/08 13:00:00,e,lowe,42768,1.00753e+06,0,260827,1.31112e+06,0,364.201,2.02334,0,0
0,n-lowe,10/08 13:00:00,n,lowe,42768,875492,0,210245,1.1285e+06,0,313.474,1.74152,0,0
0,s-lowe,10/08 11:00:00,s,lowe,53136,1.58194e+06,0,302578,1.93766e+06,0,538.238,2.99021,0,0
0,w-lowe,10/08 17:00:00,w,lowe,42768,1.86189e+06,0,424387,2.32904e+06,0,646.957,3.5942,0,0


In [80]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakElecEnergy[J](Hourly)':'L_PeakElecEnergy[J](Hourly)',
 'PeakGasEnergy[J](Hourly)':'L_PeakGasEnergy[J](Hourly)',
 'PeakElecEnergy(W)':'L_PeakElecEnergy(W)',
 'PeakElecEnergy(W/sf)':'L_PeakElecEnergy(W/sf)',
 'PeakGasEnergy(W)':'L_PeakGasEnergy(W)',
 'PeakGasEnergy(W/sf)':'L_PeakGasEnergy(W/sf)'
                            })

# allLowePeakDemandElec

In [81]:
# Reset index
allLowePeakDemandElec.reset_index(inplace=True, drop=True)

# allLowePeakDemandElec

---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [82]:
# merge existing and clear
MergedAnnualPeakElecData1 = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

# MergedAnnualPeakElecData1

In [83]:
# # #merge the above with lowe
MergedAnnualPeakElecData2 = pd.merge(MergedAnnualPeakElecData1, allLowePeakDemandElec, left_index=True, right_index=True)

# MergedAnnualPeakElecData2

In [84]:
# # Jpin two dataframes in B
# PeakFinal = pd.merge(MergedAnnualPeakElecData1, MergedAnnualPeakElecData2,  how="inner")

# PeakFinal

In [85]:
# list(MergedAnnualPeakElecData2.columns.values)

In [86]:
#Deltas of exist - clear

MergedAnnualPeakElecData2["DeltaClear_PeakElecEnergy(W)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W)"] - MergedAnnualPeakElecData2["C_PeakElecEnergy(W)"]
MergedAnnualPeakElecData2["DeltaClear_PeakElecEnergy(W/sf)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W/sf)"] - MergedAnnualPeakElecData2["C_PeakElecEnergy(W/sf)"]

# MergedAnnualPeakElecData2


In [87]:
# Deltas of exist - lowe

MergedAnnualPeakElecData2["DeltaLowe_PeakElecEnergy(W)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W)"] - MergedAnnualPeakElecData2["L_PeakElecEnergy(W)"]
MergedAnnualPeakElecData2["DeltaLowe_PeakElecEnergy(W/sf)"] = MergedAnnualPeakElecData2["B_PeakElecEnergy(W/sf)"] - MergedAnnualPeakElecData2["L_PeakElecEnergy(W/sf)"]

# MergedAnnualPeakElecData2

In [88]:
# list(MergedAnnualPeakElecData2.columns.values)

In [89]:
# Export grouped file to csv
MergedAnnualPeakElecData2.to_csv("Scraper_Output/monterey_deltas_peakElec_com.csv", header=True, index=False)